In [22]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    groq_api_key = 'yourapikey    
)

response = llm.invoke("The first person to land on moon was...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, as part of the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [23]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-49548?from=job%20search%20funnel")

docs = loader.load().pop().page_content
print(docs)

ConnectionError: HTTPSConnectionPool(host='jobs.nike.com', port=443): Max retries exceeded with url: /job/R-49548?from=job%20search%20funnel (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000216DE39D6A0>: Failed to resolve 'jobs.nike.com' ([Errno 11001] getaddrinfo failed)"))

In [16]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
    """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data' : docs})
print(res.content)
type(res.content)

```json
{
  "role": "Data Engineer I, ITC",
  "experience": "0-2 years",
  "skills": [
    "Big Data",
    "PySpark",
    "SQL",
    "Databricks",
    "Snowflake",
    "Airflow"
  ],
  "description": "Building, enhancing, and troubleshooting complex data pipelines, collaborating with product managers, engineers, analysts to build, enhance and troubleshoot data pipelines, collaborating with senior, lead and principal engineers to define and implement quality standards across data pipelines, contributing towards the design and architecture of data pipelines, implementing data quality and reliability measures across data pipelines"
}
```


str

In [17]:
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Data Engineer I, ITC',
 'experience': '0-2 years',
 'skills': ['Big Data',
  'PySpark',
  'SQL',
  'Databricks',
  'Snowflake',
  'Airflow'],
 'description': 'Building, enhancing, and troubleshooting complex data pipelines, collaborating with product managers, engineers, analysts to build, enhance and troubleshoot data pipelines, collaborating with senior, lead and principal engineers to define and implement quality standards across data pipelines, contributing towards the design and architecture of data pipelines, implementing data quality and reliability measures across data pipelines'}

In [18]:
type(json_res)

dict

In [19]:
import pandas as pd
df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [25]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [26]:
links = collection.query(query_texts=['Experience in Python', "Expertise in React"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [27]:
job = json_res
job['skills']

['Big Data', 'PySpark', 'SQL', 'Databricks', 'Snowflake', 'Airflow']

In [28]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Krishna, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Krishna, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Data Engineering Solutions for Seamless Data Pipelines

Dear Hiring Manager,

I came across the job description for a Data Engineer I, ITC, and I was impressed by the role's focus on building, enhancing, and troubleshooting complex data pipelines. As a Business Development Executive at AtliQ, I'd like to introduce our company as a potential partner in fulfilling your data engineering needs.

At AtliQ, we specialize in facilitating the seamless integration of business processes through automated tools, leveraging our expertise in AI and software consulting. Our team has a proven track record of empowering enterprises with tailored solutions, driving scalability, process optimization, cost reduction, and heightened overall efficiency.

Our capabilities align perfectly with the requirements outlined in the job description. We have extensive experience in Big Data, PySpark, SQL, Databricks, Snowflake, and Airflow, which would enable us to make significant contributions to t